## Imports

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt

## Directory, Image-Name & Image-Size Definitions

In [2]:
model_dir = '/home/drevital/cs_video_processor/models/jun22_a'
im_dir = '/home/drevital/obstacles_classification_datasets/test/eval/obstacle'
im_name = '1038_71961_0.51_.51_2022-02-01--03-13-41__unknown_.jpg'
im_height = 200
im_width = 600
target_size = (im_height, im_width)
batch_size = 32
classification = 1

## Load Trained Model

In [3]:
model = load_model(model_dir)

## Define Generator with Separate Resize

In [4]:
def three_im_generator(x, y, gen, batch_size):
    im_gen = gen.flow(x,
                      y,
                      batch_size=batch_size,
                      shuffle=False)
    im1_s, im2_s, im3_s = [], [], []
    im1, im2, im3 = [], [], []
    images, labels = im_gen.next()

    for im in images:
        w = im.shape[1]
        im1 = im[:, :w//3]
        im2 = im[:, w//3:(w*2)//3] 
        im3 = im[:, (w*2)//3:] 
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
        im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
        im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2GRAY)
        im1 = cv2.resize(im1, dsize=(200, 200), interpolation=cv2.INTER_LANCZOS4)
        im2 = cv2.resize(im2, dsize=(200, 200), interpolation=cv2.INTER_LANCZOS4)
        im3 = cv2.resize(im3, dsize=(200, 200), interpolation=cv2.INTER_NEAREST)
        #im1 = im1.reshape(1, im1.shape[0], im1.shape[1], 3)
        #im2 = im2.reshape(1, im2.shape[0], im2.shape[1], 3)
        #im3 = im3.reshape(1, im3.shape[0], im3.shape[1], 1)
        im1_s.append(im1)
        im2_s.append(im2)
        im3_s.append(im3)

    im1_s = np.array(im1_s)
    im2_s = np.array(im2_s)
    im3_s = np.array(im3_s)
    
    yield [im1_s, im2_s, im3_s], labels

## Prepare Image for Prediction

In [5]:
datagen = ImageDataGenerator(rescale=1./255.)
im_names = os.listdir(im_dir)
x = []
for im_name in im_names:
    im = cv2.imread(os.path.join(im_dir, im_name))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)  
    x.append(im)
y = [[1 for _ in range(len(x))]]
x = np.array(x, dtype='float32')
y = np.array(y, dtype='float32')
predict_generator = three_im_generator(x, y, datagen, batch_size=1)

## Predict 

In [6]:
prediction = model.predict(predict_generator)

1/1 [==============================] - 0s 465ms/step


In [7]:
prediction

array([[0.51070315]], dtype=float32)

In [8]:
x.shape

(1, 108, 366, 3)

In [9]:
y.shape

(1, 1)